In [21]:
import numpy as np
from data_helper import DatasetAndQuerysetHelper
from partition_algorithm import PartitionAlgorithm

In [22]:
used_dims = [1,2]
scale_factor=1
# block_size = int(1000000/scale_factor)
block_size = 10000
max_active_ratio=3
base_path='/home/liupengju/pycharmProjects/NORA_JOIN_SIMULATION/NORA_experiments'


In [23]:
helper = DatasetAndQuerysetHelper(used_dimensions=used_dims, scale_factor=scale_factor)

In [24]:

# helper.generate_dataset_and_save(base_path+'/dataset/lineitem_1.tbl')

In [25]:
dataset, domains = helper.load_dataset(used_dims)
boundary = [interval[0] for interval in domains] + [interval[1] for interval in domains]
print(len(dataset))  # 6001309
print(domains)

6001215
[[1.e+00 2.e+05]
 [1.e+00 1.e+04]]


In [26]:
query_amount=np.array([0.02,0.05,0.1,0.2,0.5,1])*1000
print(query_amount)

[  20.   50.  100.  200.  500. 1000.]


In [27]:
result={}
for query_num in query_amount:
    training_set, testing_set = helper.generate_queryset_and_save(int(query_num), queryset_type=1) # 1-1
    cost_res = list()
    # = = = = = Test PartitionAlgorithm (QDT) = = = = =
    pa1 = PartitionAlgorithm()
    pa1.InitializeWithQDT(training_set, len(boundary) // 2, boundary, dataset, data_threshold=block_size)

    # = = = = = Test PartitionAlgorithm (PAW) = = = = =
    pa2 = PartitionAlgorithm()
    # pa2.InitializeWithPAW(training_set, len(boundary)//2, boundary, dataset, block_size,using_beam_search=False, candidate_size = 2, candidate_depth = 2, max_active_ratio=50,strategy = 0)
    pa2.InitializeWithPAW(training_set, len(boundary) // 2, boundary, dataset, block_size, max_active_ratio=max_active_ratio,strategy=1)


    # = = = = = Test PartitionAlgorithm (ME) = = = = =
    pa3 = PartitionAlgorithm()
    # pa3.InitializeWithPAW2(training_set, len(boundary) // 2, boundary, dataset, block_size,using_beam_search=True, candidate_size = 2, candidate_depth = 2,max_active_ratio=50,strategy = 0)
    pa3.InitializeWithNORA(training_set, len(boundary)//2, boundary, dataset, data_threshold = block_size, using_beam_search=True, candidate_size = 2, candidate_depth = 2,using_1_by_1 = True, using_kd = True)
    pa3.partition_tree.set_redundant_partition(queries=training_set,data_threshold=block_size)

    algos = [pa1,pa2,pa3]
    for algo in algos:
        cost_res.append(algo.partition_tree.evaluate_query_cost(training_set, True))
    results = helper.real_result_size(dataset, training_set)
    cost_res.append(sum(results)/len(results))
    result[query_num]=np.round(np.array(cost_res)/dataset.shape[0],8)

print(result)

Build Time (s): 1.3159151077270508
Build Time (s): 2.0322675704956055
Build Time (s): 0.34600830078125
Build Time (s): 0.25565457344055176


TypeError: unsupported operand type(s) for +: 'NoneType' and 'NoneType'

In [ ]:
# search dimensionality
helper = DatasetAndQuerysetHelper(scale_factor=scale_factor) # EXAMPLE
c_dataset, c_domains = helper.load_dataset()
query_num=500
result={}
available_dims=[0,1,2,4,5]
for num_dims in range(2,6):
    # used_dims = [i for i in range(1, num_dims+1)]
    used_dims=available_dims[:num_dims]
    dataset=c_dataset[:,used_dims]
    domains=c_domains[used_dims]
    boundary = [interval[0] for interval in domains]+[interval[1] for interval in domains]
    helper.used_dimensions=used_dims
    training_set, testing_set = helper.generate_queryset_and_save(query_num, queryset_type=0)
    cost_res = list()
   # = = = = = Test PartitionAlgorithm (QDT) = = = = =
    pa1 = PartitionAlgorithm()
    pa1.InitializeWithQDT(training_set, len(boundary) // 2, boundary, dataset, data_threshold=block_size)

    # = = = = = Test PartitionAlgorithm (PAW) = = = = =
    pa2 = PartitionAlgorithm()
    # pa2.InitializeWithPAW(training_set, len(boundary)//2, boundary, dataset, block_size,using_beam_search=False, candidate_size = 2, candidate_depth = 2, max_active_ratio=50,strategy = 0)
    pa2.InitializeWithPAW(training_set, len(boundary) // 2, boundary, dataset, block_size, max_active_ratio=max_active_ratio,strategy=1)

    # = = = = = Test PartitionAlgorithm (ME) = = = = =
    pa3 = PartitionAlgorithm()
    # pa3.InitializeWithPAW2(training_set, len(boundary) // 2, boundary, dataset, block_size,using_beam_search=True, candidate_size = 2, candidate_depth = 2,max_active_ratio=50,strategy = 0)
    pa3.InitializeWithNORA(training_set, len(boundary)//2, boundary, dataset, data_threshold = block_size, using_beam_search=True, candidate_size = 2, candidate_depth = 2,using_1_by_1 = True, using_kd = True)
    pa3.partition_tree.set_redundant_partition(queries=training_set,data_threshold=block_size)

    algos = [pa1,pa2,pa3]
    for algo in algos:
        cost_res.append(algo.partition_tree.evaluate_query_cost(training_set, True))
    results = helper.real_result_size(dataset, training_set)
    cost_res.append(sum(results)/len(results))
    result[str(used_dims)]=np.round(np.array(cost_res)/dataset.shape[0],8)

print(result)
# {'[1, 2]': array([0.00604011, 0.00535958, 0.00391104, 0.00254143]), 
# '[1, 2, 3]': array([0.00314586, 0.00293257, 0.00223305, 0.00010503]), 
# '[1, 2, 3, 4]': array([2.67546e-03, 2.77061e-03, 2.23621e-03, 4.97000e-06]), 
# '[1, 2, 3, 4, 5]': array([2.53365e-03, 2.52032e-03, 2.09957e-03, 2.60000e-07])}

In [ ]:
# extended_training_set = helper.extend_queryset(training_set)

In [ ]:
# helper.visualize_queryset_and_dataset([0,1], training_set, testing_set)


In [ ]:
# distance = helper.queryset_distance(training_set[0:25], training_set[25:])
# # save the special queryset
# np.savetxt(f"{base_path}/queryset/prob1/uniform_queries_train.csv", training_set, delimiter=',')
# np.savetxt(f"{base_path}/queryset/prob1/uniform_queries_test.csv", testing_set, delimiter=',')
# np.savetxt(f"{base_path}/queryset/prob1/skewed_queries_train.csv", training_set, delimiter=',')
# np.savetxt(f"{base_path}/queryset/prob1/skewed_queries_test.csv", testing_set, delimiter=',')
